In [4]:
import numpy as np
import cv2
import handmodule as htm
import pycaw
import math

## Credits to Murtazas robotics and AI youtube channel

from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

ranged=volume.GetVolumeRange()
maxvol=ranged[0]
minvol=ranged[1]



In [5]:
detector=htm.HandDetector(detectionCon=0.69)
cap=cv2.VideoCapture(0)
cap. set(cv2.CAP_PROP_FRAME_WIDTH, 1060)
cap. set(cv2.CAP_PROP_FRAME_HEIGHT, 1020)
lmlist=[]
while True:
    _,img= cap.read()
    img = cv2.flip(img,1)
    img = detector.findhands(img)
    lmlist =  detector.findposition(img,draw=False)    
    if len(lmlist)!=0:
        x1,y1 = lmlist[4][1], lmlist[4][2]
        x2,y2 = lmlist[8][1], lmlist[8][2]
        cv2.circle(img , (x1,y1),10,(255,0,250),cv2.FILLED)
        cv2.circle(img , (x2,y2),10,(255,0,250),cv2.FILLED)
        cv2.line(img, (x1,y1),(x2,y2) ,(255,0,255) ,3)
        
        length = math.hypot(x2-x1, y2-y1)
        control = np.interp(length,[50,250],[maxvol,minvol])
        volume.SetMasterVolumeLevel(control, None)     
                                
        
        
    cv2.imshow("Volume Controller",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()